In [11]:
import numpy as np
from __future__ import print_function

### Summing Fibonacci numbers

Each new term in the Fibonacci sequence is generated by adding the
previous two terms.

By starting with 1 and 2, the first 10 terms will be:


1, 2, 3, 5, 8, 13, 21, 34, 55, 89, ...

By considering the terms in the Fibonacci sequence whose values do
not exceed four million,find the sum of the even-valued terms.

In [3]:
#1. Calculate phi
phi = (1 + np.sqrt(5)) / 2
print "Phi", phi

Phi 1.61803398875


In [4]:
#2. Find the index below 4 million
n = np.log(4 * 10 ** 6 * np.sqrt(5) + 0.5) / np.log(phi)
print "n: ",n

n:  33.2629480359


In [5]:
#3. Create an array of 1-n
n = np.arange(1, n)
print n

[  1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.  14.  15.
  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.  28.  29.  30.
  31.  32.  33.]


In [7]:
#4. Compute Fibonacci numbers
fib = (phi**n - (-1/phi) ** n) / np.sqrt(5)
print "First 9 Fibonacci Numbers: ", fib[:9]

First 9 Fibonacci Numbers:  [  1.   1.   2.   3.   5.   8.  13.  21.  34.]


In [8]:
#5. Convert to integers
fib = fib.astype(int)
print "Integers: ", fib

Integers:  [      1       1       2       3       5       8      13      21      34
      55      89     144     233     377     610     987    1597    2584
    4181    6765   10946   17711   28657   46368   75025  121393  196418
  317811  514229  832040 1346269 2178309 3524578]


In [9]:
#6. Select even-valued terms
eventerms = fib[fib % 2 == 0]
print eventerms

[      2       8      34     144     610    2584   10946   46368  196418
  832040 3524578]


In [10]:
#7. Sum the Selected terms
print eventerms.sum()

4613732


### Finding prime factors

In [15]:
#The prime factors of 13195 are 5, 7, 13 and 29.
#What is the largest prime factor of the number 600851475143 ?

N = 600851475143
LIM = 10 ** 6

def factor(n):
    #1. Create array of trial values
    a = np.ceil(np.sqrt(n))
    lim = min(n, LIM)
    
    a = np.arange(a, a + lim)
    b2 = a ** 2 - n
    
    #2. Check whether b is a square
    fractions = np.modf(np.sqrt(b2)) [0]
    
    #3. Find 0 fractions
    indices = np.where(fractions == 0)
    
    
    #4. Find the first occurence of a 0 fraction 
    a = np.ravel(np.take(a, indices))[0]   # or a = a[indices][0]
    
    a = int(a)
    b = np.sqrt(a ** 2 - n)
    b = int(b)
    c = a + b
    d = a - b
    
    if c == 1 or d == 1:
        return 
    
    print (c,d)
    factor(c)
    factor(d)
    
factor(N)

1234169 486847
1471 839
6857 71
